# Bedrock AgentCore Gatewayを使用してOpenAPI APIをMCPツールに変換する

## 概要
Bedrock AgentCore Gatewayは、お客様がインフラストラクチャやホスティングの管理を必要とせずに、既存のAPIを完全管理型MCPサーバーに変換する方法を提供します。お客様はJSON または YAML形式のOpenAPI仕様を持参できます。OAuth2で保護されたエンタープライズサポートAPIを使用するカスタマーサービスエージェントを実演します。

Gatewayワークフローでは、エージェントを外部ツールに接続するために以下のステップが含まれます：
* **Gatewayのツールを作成** - REST API用のOpenAPI仕様などのスキーマを使用してツールを定義します。OpenAPI仕様はGateway作成のためにAmazon Bedrock AgentCoreによって解析されます。
* **Gatewayエンドポイントを作成** - インバウンド認証を伴うMCPエントリーポイントとして機能するGatewayを作成します。
* **Gatewayにターゲットを追加** - Gatewayが特定のツールへのリクエストをルーティングする方法を定義するOpenAPIターゲットを設定します。OpenAPIファイルに含まれるすべてのAPIがMCP互換ツールとなり、Gateway エンドポイントURL経由で利用できるようになります。各OpenAPI GatewayターゲットでOauthを使用したアウトバウンド認証を設定します。
* **エージェントコードを更新** - エージェントをGatewayエンドポイントに接続し、統合されたMCPインターフェースを通じてすべての設定済みツールにアクセスします。

![仕組み](images/openapis-oauth-gateway.png)

### チュートリアル詳細


| 情報                 | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアル形式   | インタラクティブ                                          |
| AgentCoreコンポーネント | AgentCore Gateway, AgentCore Identity                     |
| エージェントフレームワーク | Strands Agent                                             |
| Gatewayターゲット形式  | OpenAPI                                                   |
| エージェント         | カスタマーサポートエージェント                            |
| インバウンド認証IdP  | Okta                                                      |
| アウトバウンド認証   | OAuth                                                     |
| LLMモデル            | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| チュートリアルコンポーネント | AgentCore Gatewayの作成とAgentCore Gatewayの呼び出し |
| チュートリアル分野   | 分野横断                                                  |
| 例の複雑さ           | 簡単                                                      |
| 使用SDK              | boto3                                                     |

チュートリアルの最初の部分では、いくつかのAmazonCore Gatewayターゲットを作成します

### チュートリアルアーキテクチャ
このチュートリアルでは、OpenAPI yaml/jsonファイルで定義された操作をMCPツールに変換し、Bedrock AgentCore Gatewayでホストします。
実演目的で、サポートチケットに関するクエリに答えるカスタマーサポートエージェントを構築します。エージェントはZendeskサポートAPIのOpenAPIを使用します。ソリューションはAmazon BedrockモデルとLangchain Agentを使用します。

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+を使用したJupyter notebook
* uv
* AWS認証情報
* Okta
    - client_id
    - client_secret
    - あなたのOktaドメイン（例: dev-123456.okta.com）
    - OAuth2認証サーバーID（多くの場合default）

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# SageMakerノートブックを使用しない場合はAWS認証情報を設定
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
import os
import sys

# 現在のスクリプトのディレクトリを取得
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック (例: Jupyter)

# utils.pyが含まれるディレクトリまで移動 (1レベル上)
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# sys.pathに追加
sys.path.insert(0, utils_dir)

# これでutilsをインポートできます
import utils

In [ ]:
#### Gatewayが引き受けるIAMロールを作成
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gatewayへのインバウンド認証用Oktaの設定

Okta OAuth認証機能を作成するための手順は以下です：

* [こちら](https://developer.okta.com/docs/guides/implement-grant-type/clientcreds/main/)の手順に従って、client credentialsグラント形式でアプリケーションを作成します。Oktaサブスクリプションをお持ちの場合、Admin consoleにログインし、[こちら](https://developer.okta.com/docs/guides/implement-grant-type/clientcreds/main/)で説明されている手順に従うことができます。Oktaサブスクリプションをお持ちでない場合は、無料トライアルにサインアップする必要があります。
* Admin -> Applications に移動し、秘密鍵付きクライアントを作成します。トークン要求でDemonstrating Proof of Possession (DPoP) ヘッダーを要求するを無効にします
* Okta Admin -> Security -> APIに移動します。default Authorization Serverを使用し、追加のスコープ（例: InvokeGateway）で変更します。オプションでAccess policiesとclaimsを追加できます
* Custom Scopeを定義します。認証サーバー名をクリックし、Scopesタブに移動し、"Add Scope"をクリックします
* 設定が完了したら、下記に示すようにGateway作成用のカスタムJWT Authorizerを設定するために、default Authorization Serverのメタデータ URI（別名Discovery URI）とClientID/Secretが必要です

# インバウンド認証用Okta認証機能を持つGatewayを作成

In [ ]:
import boto3
from pprint import pprint
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])

OKTA_DISCOVERY_URL="https://<YOUR OKTA DOMAIN>/oauth2/<Your app>/.well-known/openid-configuration"
OKTA_AUDIENCE="<Your audience>" # 例: MCPGateway。Oktaでの設定と一致する必要があります

auth_config = {
        "customJWTAuthorizer": {
            "allowedAudience": [OKTA_AUDIENCE],
            "discoveryUrl": OKTA_DISCOVERY_URL
        }
}
create_response = gateway_client.create_gateway(name='OpenAPIOktaGwy2',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAMロールはGateway の作成/一覧/取得/削除権限を持つ必要があります
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='Okta Authorizerを持つSDKで作成されたAgentCore Gateway'
)
pprint(create_response)
# GatewayTarget作成に使用するGatewayIDを取得
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]

# Bedrock AgentCore GatewayによってZendeskサポートAPIをMCPツールに変換

### アウトバウンド認証の認証情報プロバイダーを作成

In [ ]:
from botocore.config import Config
ZENDESK_DOMAIN="<Zendeskドメインurl>"
ZENDESK_AUTH_ENDPOINT="https://<Zendesk-domain>/oauth/authorizations/new"
ZENDESK_TOKEN_ENDPOINT="https://<Zendesk-domain>/oauth/tokens"
ZENDESK_CLIENT_ID="" # あなたのZendesk OAuth client - client id 
ZENDESK_SECRET=""  # あなたのZendesk OAuth client - client id 

sdk_config = Config(
    region_name=os.environ['AWS_DEFAULT_REGION'],
    retries={"max_attempts": 2, "mode": "standard"},
)

acps = boto3.client(
    service_name="bedrock-agentcore-control",
    config=sdk_config,
)

provider_config= {
    "customOauth2ProviderConfig": {
         "oauthDiscovery": {
             "authorizationServerMetadata": {
                 "issuer": ZENDESK_DOMAIN,
                 "authorizationEndpoint": ZENDESK_AUTH_ENDPOINT,
                 "tokenEndpoint": ZENDESK_TOKEN_ENDPOINT,
                 "responseTypes": ["token"]
             }
         },
         "clientId": ZENDESK_CLIENT_ID,
         "clientSecret": ZENDESK_SECRET
     }
 }

response = acps.create_oauth2_credential_provider(
    name="ZendeskOAuthTokenCfg", 
    credentialProviderVendor="CustomOauth2", 
    oauth2ProviderConfigInput=provider_config
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"エグレス認証情報プロバイダーARN, {credentialProviderARN}")

### OpenAPIターゲットを作成

#### ZendeskサポートOpenAPI yamlファイルをS3にアップロード

In [ ]:
# S3クライアントを作成
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWSアカウントIDとリージョンを取得
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# パラメータを定義
bucket_name = '' # OpenAPI jsonファイルをアップロードするS3バケット
file_path = 'openapi-specs/Zendesk-support-apis.yaml'
object_key = 'Zendesk-support-apis.yaml'
# put_objectを使用してファイルをアップロードし、レスポンスを読み取り
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # アカウントIDとリージョンを含むアップロードされたオブジェクトのARNを構築
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'アップロードされたオブジェクトのS3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'ファイルのアップロードエラー: {e}')

#### Gatewayターゲットを作成

OpenAPIファイル内のサーバーURLが自分のエンドポイントURLを指していることを確認してください。GatewayはOpenAPIファイルからサーバーURLを読み取り、エンドポイントを呼び出します。S3にアップロードする前に、この変更を行っていることを確認してください。

In [ ]:
# OpenAPI仕様ファイル用のS3 URI
openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

credential_config = [
    {
        "credentialProviderType" : "OAUTH",
        "credentialProvider": {
            "oauthCredentialProvider": {
                "providerArn": credentialProviderARN, 
                "scopes": ["tickets:read", "read", "tickets:write", "write"] 
            }
        }
    }
  ]

target_name="DemoOpenAPIGW"
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=target_name,
    description='SDKを使用したS3UriによるOpenAPIターゲット',
    targetConfiguration=openapi_s3_target_config,
    credentialProviderConfigurations=credential_config)

# 問題/欠陥を報告する際に含めるリクエストIDとタイムスタンプを印刷します
response_metadata = response['ResponseMetadata']

# Strands AgentからBedrock AgentCore Gatewayを呼び出し

Strands AgentはBedrock AgentCore Gatewayを通じてAWSツールとシームレスに統合され、GatewayはModel Context Protocol（MCP）仕様を実装しています。この統合により、AIエージェントとAWSサービス間での安全で標準化されたコミュニケーションが可能になります。

その核心において、Bedrock AgentCore Gatewayは基本的なMCP API（ListToolsとInvokeTools）を公開するプロトコル準拠のGatewayとして機能します。これらのAPIにより、MCP準拠のクライアントまたはSDKは、安全で標準化された方法で利用可能なツールを発見し、対話することができます。Strands AgentがAWSサービスにアクセスする必要がある場合、これらのMCP標準化エンドポイントを使用してGatewayと通信します。

Gatewayの実装は（MCP承認仕様）[https://modelcontextprotocol.org/specification/draft/basic/authorization]に厳密に準拠しており、堅牢なセキュリティとアクセス制御を確保します。これは、Strands AgentによるすべてのツールInvocationが承認ステップを通過することを意味し、強力な機能を有効にしながらセキュリティを維持します。

例えば、Strands AgentがMCPツールにアクセスする必要がある場合、まずListToolsを呼び出して利用可能なツールを発見し、次にInvokeToolsを使用して特定のアクションを実行します。Gatewayは必要なセキュリティ検証、プロトコル変換、およびサービス相互作用をすべて処理し、全体のプロセスをシームレスで安全にします。

このアーキテクチャアプローチは、MCP仕様を実装する任意のクライアントまたはSDKがGatewayを通じてAWSサービスと対話できることを意味し、AIエージェント統合のための汎用性があり将来に対応したソリューションとなります。

# インバウンド認証用Oktaからアクセストークンを要求

In [ ]:
print("Okta認証機能からアクセストークンを要求中")
import requests
from requests.auth import HTTPBasicAuth

# 実際の値に置き換えてください
OKTA_DOMAIN = "あなたのOktaドメインURL"
AUTH_SERVER_ID = "Oktaアプリid"
CLIENT_ID = "<Oktaクライアント認証情報client id>"
CLIENT_SECRET = "<Oktaクライアント認証情報secret>"

TOKEN_URL = f"{OKTA_DOMAIN}/oauth2/{AUTH_SERVER_ID}/v1/token"

response = requests.post(
    TOKEN_URL,
    auth=HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET),
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={"grant_type": "client_credentials", "scope": "InvokeGateway"}
)

if response.status_code == 200:
    token = response.json()["access_token"]
    print("Access Token:", token)
else:
    print("トークン取得失敗:", response.status_code, response.text)

# Bedrock AgentCore Gatewayを使用してZendeskサポートAPIでカスタマーサポートエージェントに質問

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentialsで設定されたIAMグループ/ユーザーはBedrockモデルへのアクセス権限が必要
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# ルートstrandsロガーを設定します。問題をデバッグする場合はDEBUGに変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを表示するためにハンドラーを追加
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # listToolsを呼び出し
    tools = client.list_tools_sync()
    # モデルとツールでAgentを作成
    agent = Agent(model=yourmodel,tools=tools) ## お好きなモデルに置き換えることができます
    #print(f"エージェントにロードされたツール: {agent.tool_names}")
    #print(f"エージェントのツール設定: {agent.tool_config}")
    
    # サンプルプロンプトでエージェントを呼び出し。これはMCP listToolsのみを呼び出し、LLMがアクセスできるツールのリストを取得します。下記では実際にツールを呼び出しません。
    #agent("こんにちは、利用可能なすべてのツールをリストしてもらえますか")
    agent("サポートチケットの数をカウントして")
    # MCPツールを明示的に呼び出し。MCPツール名と引数はAWS Lambda関数またはOpenAPI/Smithy APIと一致する必要があります
    result = client.call_tool_sync(
    tool_use_id="count-tickets-1", # これを一意の識別子に置き換えることができます。
    name="DemoOpenAPIGW___CountTickets", # これはAWS Lambda ターゲット形式に基づくツール名です。これはターゲット名に応じて変更されます
    )
    #MCPツールレスポンスを印刷
    print(f"ツール呼び出し結果: {result['content'][0]['text']}")

# クリーンアップ
IAMロール、IAMポリシー、認証情報プロバイダー、AWS Lambda関数、Cognitoユーザープール、s3バケットなどの追加リソースもクリーンアップの一環として手動で削除する必要がある場合があります。これは実行する例によって異なります。

## Gatewayを削除（オプション）

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)